## Imports and settings

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from pyfinance.ols import PandasRollingOLS

pd.set_option('display.expand_frame_repr', False)

In [22]:
sns.set_style('whitegrid')
idx = pd.IndexSlice

## Set Data Store path

In [6]:
DATA_STORE = Path('../data/assets.h5')

# Quandl wiki prices

Quandl makes available a dataset with stock prices, dividends and splits for 3000 US publicly-traded companies. Quandl decided to discontinue support in favor of its commercial offerings as of April 2018.

In [12]:
df = (pd.read_csv('../data/wiki_prices.csv',
                 parse_dates=['date'],
                 index_col=['date', 'ticker'],
                 infer_datetime_format=True)
     .sort_index())

print(df.info(null_counts=True))
with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/prices', df)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15389314 entries, (Timestamp('1962-01-02 00:00:00'), 'ARNC') to (Timestamp('2018-03-27 00:00:00'), 'ZUMZ')
Data columns (total 12 columns):
 #   Column       Non-Null Count     Dtype  
---  ------       --------------     -----  
 0   open         15388776 non-null  float64
 1   high         15389259 non-null  float64
 2   low          15389259 non-null  float64
 3   close        15389313 non-null  float64
 4   volume       15389314 non-null  float64
 5   ex-dividend  15389314 non-null  float64
 6   split_ratio  15389313 non-null  float64
 7   adj_open     15388776 non-null  float64
 8   adj_high     15389259 non-null  float64
 9   adj_low      15389259 non-null  float64
 10  adj_close    15389313 non-null  float64
 11  adj_volume   15389314 non-null  float64
dtypes: float64(12)
memory usage: 1.4+ GB
None


In [21]:
df.tail()

open   high    low  close      volume  ex-dividend  split_ratio  adj_open  adj_high  adj_low  adj_close  adj_volume
date       ticker                                                                                                                      
2018-03-27 ZIXI     4.43   4.45   4.35   4.39    207304.0          0.0          1.0      4.43      4.45     4.35       4.39    207304.0
           ZNGA     3.85   3.85   3.68   3.70  11026010.0          0.0          1.0      3.85      3.85     3.68       3.70  11026010.0
           ZOES    15.13  15.24  14.75  14.88    261242.0          0.0          1.0     15.13     15.24    14.75      14.88    261242.0
           ZTS     84.08  84.45  81.21  81.86   2676191.0          0.0          1.0     84.08     84.45    81.21      81.86   2676191.0
           ZUMZ    24.65  24.65  23.35  23.60    403884.0          0.0          1.0     24.65     24.65    23.35      23.60    403884.0

## Wiki prices metadata

In [26]:
df = pd.read_csv('../data/wiki_stocks.csv')
print(df.info(null_counts=True))

with pd.HDFStore(DATA_STORE) as store:
    store.put('quandl/wiki/stocks', df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ticker  3199 non-null   object
dtypes: object(1)
memory usage: 25.1+ KB
None


# Metadata on US-traded companies

The following downloads several attributes for [companies](https://www.nasdaq.com/screening/companies-by-name.aspx) traded on NASDAQ.

In [43]:
df = pd.read_csv('../data/nasdaq_screener.csv').dropna(how='all', axis=1)\
        .rename(columns=str.lower)\
        .rename(columns={'symbol':'ticker'})
df = df[~df.index.duplicated()]
print(df.info()) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7842 entries, 0 to 7841
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ticker      7842 non-null   object 
 1   name        7842 non-null   object 
 2   last sale   7842 non-null   object 
 3   net change  7842 non-null   float64
 4   % change    7837 non-null   object 
 5   market cap  7218 non-null   float64
 6   country     7130 non-null   object 
 7   ipo year    4545 non-null   float64
 8   volume      7842 non-null   int64  
 9   sector      5816 non-null   object 
 10  industry    5815 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 735.2+ KB
None


In [44]:
df.head()

,ticker,name,last sale,net change,% change,market cap,country,ipo year,volume,sector,industry
0,A,Agilent Technologies Inc. Common Stock,$145.01,-1.91,-1.30%,4.400230e+10,United States,1999.0,3220671,Capital Goods,Electrical Products
1,AA,Alcoa Corporation Common Stock,$32.08,-1.50,-4.467%,5.990130e+09,NaN,2016.0,9931122,Basic Industries,Metal Fabrications
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,$9.75,-0.01,-0.102%,1.218750e+09,NaN,2021.0,25931,Finance,Business Services
3,AACG,ATA Creativity Global American Depositary Shares,$2.99,0.04,1.356%,9.478655e+07,China,NaN,102857,Miscellaneous,Service to the Health Industry
4,AACQ,Artius Acquisition Inc. Class A Common Stock,$9.86,-0.12,-1.202%,8.929462e+08,United States,2020.0,3092287,Basic Industries,Major Chemicals


In [45]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('us_equities/stocks', df.set_index('ticker'))

We now load the Quandl stock price datasets covering the US equity markets 2000-18 using <code>pd.IndexSlice</code>.

In [46]:
with pd.HDFStore(DATA_STORE) as store:
    prices = (store['quandl/wiki/prices']
              .loc[idx[str(2000):str(2018), :], 'adj_close']
              .unstack('ticker'))
    stocks = store['us_equities/stocks'].loc[:, ['market cap', 'ipo year', 'sector']]

In [48]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4706 entries, 2000-01-03 to 2018-03-27
Columns: 3199 entries, A to ZUMZ
dtypes: float64(3199)
memory usage: 114.9 MB


In [49]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7842 entries, A to ZYXI
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   market cap  7218 non-null   float64
 1   ipo year    4545 non-null   float64
 2   sector      5816 non-null   object 
dtypes: float64(2), object(1)
memory usage: 245.1+ KB


## Consolidate prices and stocks info

In [50]:
stocks = stocks[~stocks.index.duplicated()]
stocks.index.name = 'ticker'

In [51]:
shared = prices.columns.intersection(stocks.index)

In [52]:
stocks = stocks.loc[shared, :]
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2036 entries, A to ZUMZ
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   market cap  2036 non-null   float64
 1   ipo year    879 non-null    float64
 2   sector      1926 non-null   object 
dtypes: float64(2), object(1)
memory usage: 63.6+ KB


In [54]:
prices = prices.loc[:, shared]
prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4706 entries, 2000-01-03 to 2018-03-27
Columns: 2036 entries, A to ZUMZ
dtypes: float64(2036)
memory usage: 73.1 MB


In [59]:
assert prices.shape[1] == stocks.shape[0]

# Feature engineering